In [ ]:
import pickle
import random

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy as sc
from scipy.spatial import Delaunay
from scipy.spatial.distance import cdist

from simplicial_kuramoto import SimplicialComplex
from simplicial_kuramoto.graph_generator import modular_graph
from simplicial_kuramoto.integrators import *
from simplicial_kuramoto.plotting import *

In [ ]:
def is_in_subspace(vector, matrix):
    ns = sc.linalg.null_space(matrix.todense())

    coeff = vector.dot(ns)
    ns_dim = coeff.shape

    vector_hat = np.zeros(ns.shape[0])

    for i in range(ns.shape[1]):
        vector_hat += coeff[i] * ns[:, i]

    error = np.sum(np.abs(vector - vector_hat))

    return error, vector_hat, coeff, ns_dim

## Two cliques no hole

In [ ]:
G = nx.Graph()

G.add_edge(0, 1, weight=1, edge_com=0)
G.add_edge(6, 7, weight=1, edge_com=0)
G.add_edge(7, 0, weight=1, edge_com=0)
G.add_edge(0, 6, weight=1, edge_com=0)
G.add_edge(1, 7, weight=1, edge_com=0)

G.add_edge(1, 6, weight=1, edge_com=0)

G.add_edge(2, 3, weight=1, edge_com=1)
G.add_edge(3, 4, weight=1, edge_com=1)
G.add_edge(4, 5, weight=1, edge_com=1)
G.add_edge(2, 4, weight=1, edge_com=1)
G.add_edge(3, 5, weight=1, edge_com=1)

G.add_edge(2, 5, weight=1, edge_com=1)

# G.add_edge(1,2,weight=1,edge_com=2)
G.add_edge(5, 6, weight=1, edge_com=2)

node_com_dict = dict(
    zip(list(np.linspace(0, 7, 8).astype(int)), [0, 0, 1, 1, 1, 1, 0, 0])
)
nx.set_node_attributes(G, node_com_dict, "node_com")

edge_community_assignment = np.array(
    list(nx.get_edge_attributes(G, "edge_com").values())
)


Gsc = SimplicialComplex(graph=G, no_faces=False)

plt.figure()
nx.draw_networkx(G)
plt.show()

In [ ]:
# np.random.seed(4444)
initial_phase = np.random.uniform(0, 1, Gsc.n_edges)

t_max = 50
n_t = 100
alpha_1 = np.ones(Gsc.lifted_N1.shape[1]) * 0
alpha_2 = np.ones(Gsc.lifted_N1.shape[0]) * 0
# alpha_2=np.random.uniform(0, 2 * np.pi, Gsc.lifted_N1.shape[0]) # np.ones(Gsc.lifted_N1.shape[0])*np.pi

initial_phase = np.hstack([initial_phase, -initial_phase])

edge_result = integrate_edge_kuramoto(
    Gsc, initial_phase, t_max, n_t, alpha_1=alpha_1, alpha_2=alpha_2
)
plot_edge_kuramoto(edge_result)

In [ ]:
op = module_order_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.figure()
plt.plot(op.T)
# plt.ylim([0,1])
plt.legend(["0", "1", "2", "tot"])
plt.show

## Two cliques one hole

In [ ]:
G = nx.Graph()

G.add_edge(0, 1, weight=1, edge_com=0)
G.add_edge(6, 7, weight=1, edge_com=0)
G.add_edge(7, 0, weight=1, edge_com=0)
G.add_edge(0, 6, weight=1, edge_com=0)
G.add_edge(1, 7, weight=1, edge_com=0)

G.add_edge(1, 6, weight=1, edge_com=2)

G.add_edge(2, 3, weight=1, edge_com=1)
G.add_edge(3, 4, weight=1, edge_com=1)
G.add_edge(4, 5, weight=1, edge_com=1)
G.add_edge(2, 4, weight=1, edge_com=1)
G.add_edge(3, 5, weight=1, edge_com=1)

G.add_edge(2, 5, weight=1, edge_com=2)

G.add_edge(1, 2, weight=1, edge_com=2)
G.add_edge(5, 6, weight=1, edge_com=2)

node_com_dict = dict(
    zip(list(np.linspace(0, 7, 8).astype(int)), [0, 0, 1, 1, 1, 1, 0, 0])
)
nx.set_node_attributes(G, node_com_dict, "node_com")

edge_community_assignment = np.array(
    list(nx.get_edge_attributes(G, "edge_com").values())
)


Gsc = SimplicialComplex(graph=G, no_faces=False)

plt.figure()
nx.draw_networkx(G)
plt.show()

In [ ]:
# np.random.seed(4444)
initial_phase = np.random.uniform(0, np.pi, Gsc.n_edges)
# initial_phase = np.random.normal(0, 0.2, Gsc.n_edges)

t_max = 500
n_t = 100
alpha_1 = np.ones(Gsc.lifted_N1.shape[1]) * 0.1
alpha_2 = np.ones(Gsc.lifted_N1.shape[0]) * 0
# alpha_2=np.random.uniform(0, 2 * np.pi, Gsc.lifted_N1.shape[0]) # np.ones(Gsc.lifted_N1.shape[0])*np.pi

initial_phase = np.hstack([initial_phase, -initial_phase])

edge_result = integrate_edge_kuramoto(
    Gsc, initial_phase, t_max, n_t, alpha_1=alpha_1, alpha_2=alpha_2
)
plot_edge_kuramoto(edge_result)

In [ ]:
op = module_order_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.figure()
plt.plot(op.T)
# plt.ylim([0,1])
plt.legend(["0", "1", "2", "tot"])
plt.show

In [ ]:
theta = np.round(edge_result.y[:, -1] + np.pi, 2) % (2 * np.pi) - np.pi
error, theta_hat, coeff, dim = is_in_subspace(theta, Gsc.lifted_L1)
print(error)
print(coeff)
print(dim)

plt.figure()
plt.plot(theta)
plt.plot(theta_hat, "o")
# plt.legend('observed state','reconstructed in null-space')
plt.show()

In [ ]:
plt.figure()

In [ ]:
print(Gsc.lifted_L1.dot(edge_result.y[:, -10]))
print(Gsc.L1.dot(edge_result.y[0:14:, -1]))
print(Gsc.V.T.dot(Gsc.lifted_L1.dot(Gsc.V)).dot(edge_result.y[0:14:, -10]))
print()

## No hole square

In [ ]:
G = nx.Graph()

G.add_edge(0, 1, weight=1, edge_com=0)
G.add_edge(1, 2, weight=1, edge_com=0)
G.add_edge(2, 3, weight=1, edge_com=0)
G.add_edge(3, 0, weight=1, edge_com=0)
G.add_edge(0, 2, weight=1, edge_com=0)

node_com_dict = dict(zip(list(np.linspace(0, 7, 8).astype(int)), [0, 0, 0, 0]))
nx.set_node_attributes(G, node_com_dict, "node_com")

edge_community_assignment = np.array(
    list(nx.get_edge_attributes(G, "edge_com").values())
)

Gsc = SimplicialComplex(graph=G, no_faces=False)

plt.figure()
nx.draw_networkx(G)
plt.show()

In [ ]:
initial_phase = np.random.uniform(0, 1, Gsc.n_edges)
initial_phase = np.random.normal(0, 2, Gsc.n_edges)

t_max = 50
n_t = 100
alpha_1 = np.ones(Gsc.lifted_N1.shape[1]) * 0.1
alpha_2 = np.ones(Gsc.lifted_N1.shape[0]) * 0

initial_phase = np.hstack([initial_phase, -initial_phase])

edge_result = integrate_edge_kuramoto(
    Gsc, initial_phase, t_max, n_t, alpha_1=alpha_1, alpha_2=alpha_2
)
plot_edge_kuramoto(edge_result)

In [ ]:
op = module_order_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.figure()
plt.plot(op.T)
# plt.ylim([0,1])
plt.legend(["0", "tot"])
plt.show

In [ ]:
plt.figure()
plt.plot(edge_result.y[:, :].T)
# plt.plot(edge_result.y[7,:].T)
plt.show()

## One hole square

In [ ]:
G = nx.Graph()

G.add_edge(0, 1, weight=1, edge_com=0)
G.add_edge(1, 2, weight=1, edge_com=0)
G.add_edge(2, 3, weight=1, edge_com=0)
G.add_edge(3, 0, weight=1, edge_com=0)

node_com_dict = dict(zip(list(np.linspace(0, 3, 4).astype(int)), [0, 0, 0, 0]))
nx.set_node_attributes(G, node_com_dict, "node_com")

edge_community_assignment = np.array(
    list(nx.get_edge_attributes(G, "edge_com").values())
)

Gsc = SimplicialComplex(graph=G, no_faces=False)

plt.figure()
nx.draw_networkx(G)
plt.show()

In [ ]:
initial_phase = np.random.uniform(2, 2.1, Gsc.n_edges)
# initial_phase = np.random.normal(0, 2, Gsc.n_edges)

t_max = 500
n_t = 200
alpha_1 = np.ones(Gsc.lifted_N1.shape[1]) * 0.1
alpha_2 = np.ones(Gsc.lifted_N1.shape[0]) * 0

initial_phase = np.hstack([initial_phase, -initial_phase])

edge_result = integrate_edge_kuramoto(
    Gsc, initial_phase, t_max, n_t, alpha_1=alpha_1, alpha_2=alpha_2
)
plot_edge_kuramoto(edge_result)

In [ ]:
plt.figure()
plt.plot(edge_result.y[:, :].T)
# plt.plot(edge_result.y[7,:].T)
plt.show()

In [ ]:
op = module_order_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.figure()
plt.plot(op.T)
# plt.ylim([0,1])
plt.legend(["0", "tot"])
plt.show

In [ ]:
gop, phase_gradient = module_gradient_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.imshow(phase_gradient)
plt.figure()
plt.plot(gop.T)
plt.show

print(coalition_entropy(gop))

In [ ]:
theta = np.round(edge_result.y[:, -20] + np.pi, 2) % (2 * np.pi) - np.pi
error, theta_hat, coeff, dim = is_in_subspace(theta, Gsc.lifted_L1)
print(error)
print(coeff)
print(dim)

plt.figure()
plt.plot(theta)
plt.plot(theta_hat, "o")
# plt.legend('observed state','reconstructed in null-space')
plt.show()

theta = edge_result.y[0:4, -1]
error, theta_hat, coeff, dim = is_in_subspace(theta, Gsc.L1)
print(error)
print(coeff)
print(dim)

plt.figure()
plt.plot(theta)
plt.plot(theta_hat, "o")
plt.show()

In [ ]:
print(Gsc.lifted_L1.dot(edge_result.y[:, -10]))
print(Gsc.L1.dot(edge_result.y[0:4:, -10]))
print(Gsc.V.T.dot(Gsc.lifted_L1.dot(Gsc.V)).dot(edge_result.y[0:4:, -10]))

In [ ]:
print(np.linalg.eigvals(Gsc.L1.todense()))

plt.figure()
plt.imshow(Gsc.V.dot(sc.linalg.null_space(Gsc.L1.todense())))
plt.show()

print(np.linalg.eigvals(Gsc.lifted_L1.todense()))

plt.figure()
plt.imshow(np.real(sc.linalg.null_space(Gsc.lifted_L1.todense())))
plt.title("")
plt.show()

## Hexagone

In [ ]:
G = nx.Graph()

G.add_edge(0, 1, weight=1, edge_com=0)
G.add_edge(1, 2, weight=1, edge_com=1)
G.add_edge(2, 3, weight=1, edge_com=2)
G.add_edge(3, 4, weight=1, edge_com=0)
G.add_edge(4, 5, weight=1, edge_com=1)
G.add_edge(5, 0, weight=1, edge_com=2)

node_com_dict = dict(zip(list(np.linspace(0, 5, 6).astype(int)), [0, 0, 0, 0, 0, 0]))
nx.set_node_attributes(G, node_com_dict, "node_com")

edge_community_assignment = np.array(
    list(nx.get_edge_attributes(G, "edge_com").values())
)

Gsc = SimplicialComplex(graph=G, no_faces=False)

plt.figure()
nx.draw_networkx(G)
plt.show()

In [ ]:
initial_phase = np.random.uniform(0, 1, Gsc.n_edges)
# initial_phase = np.random.normal(0, 2, Gsc.n_edges)

t_max = 500
n_t = 200
alpha_1 = np.ones(Gsc.lifted_N1.shape[1]) * 0.1
alpha_2 = np.ones(Gsc.lifted_N1.shape[0]) * 0

initial_phase = np.hstack([initial_phase, -initial_phase])

edge_result = integrate_edge_kuramoto(
    Gsc, initial_phase, t_max, n_t, alpha_1=alpha_1, alpha_2=alpha_2
)
plot_edge_kuramoto(edge_result)

In [ ]:
plt.figure()
plt.plot(edge_result.y[:, :].T)
# plt.plot(edge_result.y[7,:].T)
plt.show()

In [ ]:
op = module_order_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.figure()
plt.plot(op.T)
# plt.ylim([0,1])
plt.legend(["0", "tot"])
plt.show

In [ ]:
gop, phase_gradient = module_gradient_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.imshow(phase_gradient)
plt.figure()
plt.plot(gop.T)
plt.show

In [ ]:
theta = np.round(edge_result.y[:, -1] + np.pi, 2) % (2 * np.pi) - np.pi
error, theta_hat, coeff, dim = is_in_subspace(theta, Gsc.lifted_L1)
print(error)
print(coeff)
print(dim)

plt.figure()
plt.plot(theta)
plt.plot(theta_hat, "o")
# plt.legend('observed state','reconstructed in null-space')
plt.show()

## Little house

In [ ]:
G = nx.Graph()

G.add_edge(0, 1, weight=1, edge_com=0)
G.add_edge(1, 2, weight=1, edge_com=0)
G.add_edge(2, 3, weight=1, edge_com=0)
G.add_edge(3, 4, weight=1, edge_com=0)
G.add_edge(4, 0, weight=1, edge_com=0)
# G.add_edge(5,0,weight=1,edge_com=0)

G.add_edge(2, 0, weight=1, edge_com=0)

node_com_dict = dict(zip(list(np.linspace(0, 5, 6).astype(int)), [0, 0, 0, 0, 0, 0]))
nx.set_node_attributes(G, node_com_dict, "node_com")

edge_community_assignment = np.array(
    list(nx.get_edge_attributes(G, "edge_com").values())
)

Gsc = SimplicialComplex(graph=G, no_faces=False)

plt.figure()
nx.draw_networkx(G)
plt.show()

In [ ]:
initial_phase = np.random.uniform(0, 0.1, Gsc.n_edges)
# initial_phase = np.random.normal(0, 2, Gsc.n_edges)

t_max = 400
n_t = 100
alpha_1 = np.ones(Gsc.lifted_N1.shape[1]) * 0.1
alpha_2 = np.ones(Gsc.lifted_N1.shape[0]) * 0

initial_phase = np.hstack([initial_phase, -initial_phase])

edge_result = integrate_edge_kuramoto(
    Gsc, initial_phase, t_max, n_t, alpha_1=alpha_1, alpha_2=alpha_2
)
plot_edge_kuramoto(edge_result)

In [ ]:
plt.figure()
plt.plot(edge_result.y[:, :].T)
# plt.plot(edge_result.y[7,:].T)
plt.show()

In [ ]:
op = module_order_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.figure()
plt.plot(op.T)
# plt.ylim([0,1])
plt.legend(["0", "tot"])
plt.show

In [ ]:
gop, phase_gradient = module_gradient_parameter(
    edge_result.y, np.hstack([edge_community_assignment, edge_community_assignment])
)
plt.imshow(phase_gradient)
plt.figure()
plt.plot(gop.T)
plt.show

In [ ]:
theta = np.round(edge_result.y[:, -1] + np.pi, 2) % (2 * np.pi) - np.pi
error, theta_hat, coeff, dim = is_in_subspace(theta, Gsc.lifted_L1)
print(error)
print(coeff)
print(dim)

plt.figure()
plt.plot(theta)
plt.plot(theta_hat, "o")
# plt.legend('observed state','reconstructed in null-space')
plt.show()